# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa250a53820>
├── 'a' --> tensor([[-0.6577,  1.1266,  0.0993],
│                   [ 0.3576,  0.0851, -0.7748]])
└── 'x' --> <FastTreeValue 0x7fa250a53130>
    └── 'c' --> tensor([[ 0.4248,  0.2888,  0.8925, -0.6063],
                        [ 0.2922, -0.0568,  1.1804,  1.5707],
                        [ 0.2536, -0.8390,  0.1134, -0.9493]])

In [4]:
t.a

tensor([[-0.6577,  1.1266,  0.0993],
        [ 0.3576,  0.0851, -0.7748]])

In [5]:
%timeit t.a

63.8 ns ± 0.036 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa250a53820>
├── 'a' --> tensor([[-1.6051, -0.9538, -1.8929],
│                   [ 0.0312, -1.6531, -0.9355]])
└── 'x' --> <FastTreeValue 0x7fa250a53130>
    └── 'c' --> tensor([[ 0.4248,  0.2888,  0.8925, -0.6063],
                        [ 0.2922, -0.0568,  1.1804,  1.5707],
                        [ 0.2536, -0.8390,  0.1134, -0.9493]])

In [7]:
%timeit t.a = new_value

63.3 ns ± 0.0263 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6577,  1.1266,  0.0993],
               [ 0.3576,  0.0851, -0.7748]]),
    x: Batch(
           c: tensor([[ 0.4248,  0.2888,  0.8925, -0.6063],
                      [ 0.2922, -0.0568,  1.1804,  1.5707],
                      [ 0.2536, -0.8390,  0.1134, -0.9493]]),
       ),
)

In [10]:
b.a

tensor([[-0.6577,  1.1266,  0.0993],
        [ 0.3576,  0.0851, -0.7748]])

In [11]:
%timeit b.a

55.5 ns ± 0.0282 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.5104,  0.3684, -0.6002],
               [ 0.2033,  0.6438, -0.3357]]),
    x: Batch(
           c: tensor([[ 0.4248,  0.2888,  0.8925, -0.6063],
                      [ 0.2922, -0.0568,  1.1804,  1.5707],
                      [ 0.2536, -0.8390,  0.1134, -0.9493]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.25 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

869 ns ± 0.22 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 33.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 243 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 376 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa1a571d880>
├── 'a' --> tensor([[[-0.6577,  1.1266,  0.0993],
│                    [ 0.3576,  0.0851, -0.7748]],
│           
│                   [[-0.6577,  1.1266,  0.0993],
│                    [ 0.3576,  0.0851, -0.7748]],
│           
│                   [[-0.6577,  1.1266,  0.0993],
│                    [ 0.3576,  0.0851, -0.7748]],
│           
│                   [[-0.6577,  1.1266,  0.0993],
│                    [ 0.3576,  0.0851, -0.7748]],
│           
│                   [[-0.6577,  1.1266,  0.0993],
│                    [ 0.3576,  0.0851, -0.7748]],
│           
│                   [[-0.6577,  1.1266,  0.0993],
│                    [ 0.3576,  0.0851, -0.7748]],
│           
│                   [[-0.6577,  1.1266,  0.0993],
│                    [ 0.3576,  0.0851, -0.7748]],
│           
│                   [[-0.6577,  1.1266,  0.0993],
│                    [ 0.3576,  0.0851, -0.7748]]])
└── 'x' --> <FastTreeValue 0x7fa1a571d310>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 30.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa1a573bf40>
├── 'a' --> tensor([[-0.6577,  1.1266,  0.0993],
│                   [ 0.3576,  0.0851, -0.7748],
│                   [-0.6577,  1.1266,  0.0993],
│                   [ 0.3576,  0.0851, -0.7748],
│                   [-0.6577,  1.1266,  0.0993],
│                   [ 0.3576,  0.0851, -0.7748],
│                   [-0.6577,  1.1266,  0.0993],
│                   [ 0.3576,  0.0851, -0.7748],
│                   [-0.6577,  1.1266,  0.0993],
│                   [ 0.3576,  0.0851, -0.7748],
│                   [-0.6577,  1.1266,  0.0993],
│                   [ 0.3576,  0.0851, -0.7748],
│                   [-0.6577,  1.1266,  0.0993],
│                   [ 0.3576,  0.0851, -0.7748],
│                   [-0.6577,  1.1266,  0.0993],
│                   [ 0.3576,  0.0851, -0.7748]])
└── 'x' --> <FastTreeValue 0x7fa1a5d4f940>
    └── 'c' --> tensor([[ 0.4248,  0.2888,  0.8925, -0.6063],
                        [ 0.2922, -0.0568,  1.1804,  1.5707],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 63.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.3 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.6577,  1.1266,  0.0993],
                [ 0.3576,  0.0851, -0.7748]],
       
               [[-0.6577,  1.1266,  0.0993],
                [ 0.3576,  0.0851, -0.7748]],
       
               [[-0.6577,  1.1266,  0.0993],
                [ 0.3576,  0.0851, -0.7748]],
       
               [[-0.6577,  1.1266,  0.0993],
                [ 0.3576,  0.0851, -0.7748]],
       
               [[-0.6577,  1.1266,  0.0993],
                [ 0.3576,  0.0851, -0.7748]],
       
               [[-0.6577,  1.1266,  0.0993],
                [ 0.3576,  0.0851, -0.7748]],
       
               [[-0.6577,  1.1266,  0.0993],
                [ 0.3576,  0.0851, -0.7748]],
       
               [[-0.6577,  1.1266,  0.0993],
                [ 0.3576,  0.0851, -0.7748]]]),
    x: Batch(
           c: tensor([[[ 0.4248,  0.2888,  0.8925, -0.6063],
                       [ 0.2922, -0.0568,  1.1804,  1.5707],
                       [ 0.2536, -0.8390,  0.1134, -0.9493]],
         

In [26]:
%timeit Batch.stack(batches)

78.1 µs ± 88.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.6577,  1.1266,  0.0993],
               [ 0.3576,  0.0851, -0.7748],
               [-0.6577,  1.1266,  0.0993],
               [ 0.3576,  0.0851, -0.7748],
               [-0.6577,  1.1266,  0.0993],
               [ 0.3576,  0.0851, -0.7748],
               [-0.6577,  1.1266,  0.0993],
               [ 0.3576,  0.0851, -0.7748],
               [-0.6577,  1.1266,  0.0993],
               [ 0.3576,  0.0851, -0.7748],
               [-0.6577,  1.1266,  0.0993],
               [ 0.3576,  0.0851, -0.7748],
               [-0.6577,  1.1266,  0.0993],
               [ 0.3576,  0.0851, -0.7748],
               [-0.6577,  1.1266,  0.0993],
               [ 0.3576,  0.0851, -0.7748]]),
    x: Batch(
           c: tensor([[ 0.4248,  0.2888,  0.8925, -0.6063],
                      [ 0.2922, -0.0568,  1.1804,  1.5707],
                      [ 0.2536, -0.8390,  0.1134, -0.9493],
                      [ 0.4248,  0.2888,  0.8925, -0.6063],
                      [ 0.2922, -

In [28]:
%timeit Batch.cat(batches)

146 µs ± 203 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
